In [1]:
import datetime
import ee
import pandas as pd
import sys
from IPython.display import Image
import geopandas as gpd 

sys.path.append(r'C:\openet_brazil\openet-ssebop')
import openet.ssebop as model

ee.Initialize()

In [2]:
towers_info = pd.read_csv(r"C:\openet_brazil\flux-towers-data-br\station_info.csv",engine='python',encoding='latin-1')

towers_info.head()

,local_abbr,site_id,country,site_name,has_data,all_seb_fluxes,missing_seb_data,all_met_data,missing_met_data,date_parser,...,rel_humidity_col,rel_humidity_units,wind_spd_col,wind_spd_units,wind_dir_col,wind_dir_units,theta_1,theta_1_units,theta_2,theta_2_units
0,ACE,ACE,BR,Aceguá,0.0,1.0,NaN,NaN,NaN,%Y%m%d%H%M,...,RH,"(%): Relative humidity, range 0-100",WS,m/s,WD,azimuth (degrees),SWC_1_1_1,"(%): Soil water content (volumetric), range 0-100",SWC_1_2_1,"(%): Soil water content (volumetric), range 0-100"
1,AFL,AFL,BR,Alta Floresta,1.0,0.0,G,0.0,"WD, WS, VPD, Ustar, Pair",%Y%m%d%H%M,...,RH,"(%): Relative humidity, range 0-100",WS,m/s,WD,azimuth (degrees),SWC_1_1_1,"(%): Soil water content (volumetric), range 0-100",SWC_1_2_1,"(%): Soil water content (volumetric), range 0-100"
2,BAN,BAN,BR,Javaés/Tocantins (Bananal),1.0,1.0,NaN,1.0,"VPD, Ustar",%Y%m%d%H%M,...,RH,"(%): Relative humidity, range 0-100",WS,m/s,WD,azimuth (degrees),SWC_1_1_1,"(%): Soil water content (volumetric), range 0-100",SWC_1_2_1,"(%): Soil water content (volumetric), range 0-100"
3,BDP,BDP,BR,Baía das Pedras Experimental,1.0,1.0,NaN,0.0,"WD, VPD, Ustar, Pair",%Y%m%d%H%M,...,RH,"(%): Relative humidity, range 0-100",WS,m/s,WD,azimuth (degrees),SWC_1_1_1,"(%): Soil water content (volumetric), range 0-100",SWC_1_2_1,"(%): Soil water content (volumetric), range 0-100"
4,BRA,BRA,BR,Brasília,0.0,0.0,NaN,NaN,NaN,%Y%m%d%H%M,...,RH,"(%): Relative humidity, range 0-100",WS,m/s,WD,azimuth (degrees),SWC_1_1_1,"(%): Soil water content (volumetric), range 0-100",SWC_1_2_1,"(%): Soil water content (volumetric), range 0-100"


In [6]:


for tower_name in towers_info['site_id']:

    try:
        tower_info = towers_info[towers_info['site_id'] == tower_name.upper()]

        # Date range you want to aggregate ET over
        start_date = str(tower_info['start_date'].values[0])[:4] +'-'+\
                    str(tower_info['start_date'].values[0])[4:6]+'-'+\
                    str(tower_info['start_date'].values[0])[6:8]

        end_date = str(tower_info['end_date'].values[0])[:4] +'-'+\
                    str(tower_info['end_date'].values[0])[4:6]+'-'+\
                    str(tower_info['end_date'].values[0])[6:8]

        # geometry location
        lat = tower_info['station_latitude'].values[0]
        lon = tower_info['station_longitude'].values[0]

        collections = [ 
            'LANDSAT/LT04/C02/T1_L2',
            'LANDSAT/LT05/C02/T1_L2',
            'LANDSAT/LE07/C02/T1_L2',
            'LANDSAT/LC08/C02/T1_L2',
            'LANDSAT/LC09/C02/T1_L2']

        # Only keep images with an average cloud cover less than
        # Cloud cover filter parameter is not being passed in (yet)
        cloud_cover = 30

        test_point = ee.Geometry.Point([lon, lat])



        model_obj = model.Collection(
            collections=collections,
            start_date=start_date,
            end_date=end_date,
            geometry=test_point,
            cloud_cover_max=cloud_cover,
            et_reference_source='projects/et-brasil/assets/ERA5/daily/DAILY_ETO',
            et_reference_band='eto',

        )

        et_col = model_obj.overpass(variables=[ 'et'])

        print(tower_name+ ' num images:',len(model_obj.get_image_ids()))

        #model_obj.get_image_ids()

        def get_table(img):
            
            reduce= img.reduceRegion(
                reducer=ee.Reducer.mean(),
                scale=90,
                maxPixels=1e12,
                geometry=test_point)

            date= ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
            
            return ee.Feature(None, {
            'date' : date,
            'et': ee.List([reduce.get('et'),-9999]).reduce(ee.Reducer.firstNonNull()),
            })

        # Export the image sample feature collection to Drive as a CSV file.
        task = ee.batch.Export.table.toDrive(
            collection=et_col.map(get_table),
            description='ssebop-opnenet_'+str(tower_name.lower()),
            folder='ssebop-python-openet-br',
            fileFormat='CSV',
        )
        task.start()
    except:
        print('ops, an error occured with the tower ',tower_name)
            

ACE num images: 0
AFL num images: 11
BAN num images: 55
BDP num images: 183
BRA num images: 0
CAM num images: 192
CAS num images: 78
CAX num images: 18
CMT num images: 0
CRA num images: 92
ops, an error occured with the tower  CRZ
CSE1 num images: 96
CSE2 num images: 83
CSJ num images: 22
CST num images: 239
FEX num images: 74
FMI num images: 80
FNS num images: 93
FSN num images: 27
GUY num images: 59
K34 num images: 76
K67 num images: 73
K77 num images: 40
K83 num images: 22
NPW num images: 209
PAN num images: 1
PAS num images: 48
PDG num images: 53
PRS num images: 19
QFR num images: 16
RJA num images: 43
SIN num images: 101
SJA num images: 19
SLU num images: 64
SMA num images: 192
TF1 num images: 22
TF2 num images: 5
TNR num images: 21
USE num images: 0
VIR num images: 204
ops, an error occured with the tower  nan


In [6]:
str(tower_name).lower()

'nan'